# Homework Session 11 Neural networks

## Part 1

The purpose of the below is to classify days over years 2017-2018 by their corresponding mobility patterns between 10 zones in Taipei (quantified by an aggregated temporal network of subway ridership flows across the city)

In [38]:
import numpy as np
import pandas as pd
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from keras import backend as K
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import heapq

In [39]:
#read the data
TNet=pd.read_csv('https://raw.githubusercontent.com/CUSP2021ADS/Data/main/taipeiD_TNet2.csv',header=None);

In [40]:
TNet.head() 
#each row represents a 10x10 adjacency matrix of the normalized Taipei subway mobility network between 10 zones flattened into a 100x1 row corresponding to a single day
#days start at jan-1-2017

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.017943,0.005415,0.003590,0.008316,0.007859,0.012942,0.012196,0.019543,0.001196,0.003327,...,0.002529,0.001533,0.001860,0.002375,0.005408,0.008922,0.003945,0.011075,0.005073,0.012708
1,0.021283,0.005215,0.003530,0.009359,0.007803,0.014288,0.011185,0.019044,0.001382,0.003499,...,0.002803,0.001757,0.001783,0.002549,0.005515,0.009650,0.003596,0.009618,0.005946,0.013709
2,0.028988,0.006511,0.005591,0.012970,0.007816,0.015878,0.010973,0.015768,0.002252,0.005388,...,0.004649,0.002555,0.002672,0.004291,0.007385,0.009558,0.004293,0.008791,0.010040,0.016301
3,0.029534,0.006471,0.005615,0.013017,0.007717,0.016098,0.011182,0.015815,0.002325,0.005443,...,0.004611,0.002473,0.002636,0.004195,0.007255,0.009487,0.004316,0.008729,0.010296,0.016437
4,0.029333,0.006525,0.005727,0.013098,0.007692,0.016358,0.011000,0.015677,0.002344,0.005527,...,0.004694,0.002515,0.002677,0.004222,0.007269,0.009921,0.004387,0.008923,0.010381,0.016914


In [41]:
#convert to an array and scale the data
X=np.array(TNet);

In [42]:
X=MinMaxScaler(feature_range=(0, 1), copy=True).fit_transform(X)

In [43]:
X.shape

(669, 100)

In [44]:
#define day of the week corresponding to each day of observation; 0-Sunday, 1-Monday,...,6-Saturday
y=np.array(range(669))%7; y[:10]

array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2])

In [45]:
yc=np_utils.to_categorical(y) #get categorical binary variables isSunday, isMonday,...
yc[:5]

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [46]:
X_test=X[400:,:]; X_train=X[:400,:]; #split the data into training and test
y_test=yc[400:,:]; y_train=yc[:400,:]

## Task 1. Classify weekdays/weekends
Label the rows with ones for weekends, zeros for weekdays.
Train a neural network with 4 layers of 30,10,3 and 1 (output) neurons over the training sample against this label, evaluating its performance over the test sample. Report the acheived accuracy (categorical) over the test sample

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 100 epochs, batch_size=20. 

In [47]:
def ConvertWeekend(lst):
    Y_weekend = []
    for i in lst:
        if i[0] == 1 or i[6] == 1:
            Y_weekend.append(1)
        else:
            Y_weekend.append(0)
    Y_weekend = np.asarray(Y_weekend)
    return Y_weekend
Y_wkd_train = ConvertWeekend(y_train)
Y_wkd_train.shape

(400,)

In [48]:
dim = X_train.shape[1]

In [49]:
Y_wkd_test = ConvertWeekend(y_test)
Y_wkd_test.shape

(269,)

In [50]:
model1 = Sequential()
model1.add(Dense(30, activation='relu', input_dim=dim))
model1.add(Dense(10, activation='relu'))
model1.add(Dense(3, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.fit(X_train, Y_wkd_train, validation_data=(X_test, Y_wkd_test), epochs=100, batch_size=20, verbose=2)

Epoch 1/100
20/20 - 1s - loss: 0.5836 - accuracy: 0.6650 - val_loss: 0.4780 - val_accuracy: 0.7286
Epoch 2/100
20/20 - 0s - loss: 0.4130 - accuracy: 0.7200 - val_loss: 0.3373 - val_accuracy: 0.7584
Epoch 3/100
20/20 - 0s - loss: 0.2822 - accuracy: 0.8825 - val_loss: 0.2066 - val_accuracy: 0.9554
Epoch 4/100
20/20 - 0s - loss: 0.1788 - accuracy: 0.9550 - val_loss: 0.1475 - val_accuracy: 0.9554
Epoch 5/100
20/20 - 0s - loss: 0.1429 - accuracy: 0.9550 - val_loss: 0.1273 - val_accuracy: 0.9554
Epoch 6/100
20/20 - 0s - loss: 0.1289 - accuracy: 0.9550 - val_loss: 0.1218 - val_accuracy: 0.9554
Epoch 7/100
20/20 - 0s - loss: 0.1210 - accuracy: 0.9550 - val_loss: 0.1148 - val_accuracy: 0.9591
Epoch 8/100
20/20 - 0s - loss: 0.1158 - accuracy: 0.9575 - val_loss: 0.1128 - val_accuracy: 0.9554
Epoch 9/100
20/20 - 0s - loss: 0.1126 - accuracy: 0.9600 - val_loss: 0.1113 - val_accuracy: 0.9591
Epoch 10/100
20/20 - 0s - loss: 0.1044 - accuracy: 0.9525 - val_loss: 0.1078 - val_accuracy: 0.9628
Epoch 11/

20/20 - 0s - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0822 - val_accuracy: 0.9703
Epoch 84/100
20/20 - 0s - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.0868 - val_accuracy: 0.9703
Epoch 85/100
20/20 - 0s - loss: 0.0042 - accuracy: 1.0000 - val_loss: 0.0877 - val_accuracy: 0.9703
Epoch 86/100
20/20 - 0s - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.0819 - val_accuracy: 0.9740
Epoch 87/100
20/20 - 0s - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0973 - val_accuracy: 0.9665
Epoch 88/100
20/20 - 0s - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.0840 - val_accuracy: 0.9703
Epoch 89/100
20/20 - 0s - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.0875 - val_accuracy: 0.9703
Epoch 90/100
20/20 - 0s - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.0896 - val_accuracy: 0.9703
Epoch 91/100
20/20 - 0s - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.0941 - val_accuracy: 0.9703
Epoch 92/100
20/20 - 0s - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0913 - val_accuracy: 0.9703
Epoch 93/100


In [51]:
result = pd.DataFrame()
result['Y_wkd_test'] = Y_wkd_test
result['dl_predict'] = model1.predict(X_test)
result['dl_label']=1.0*(result['dl_predict']>0.5)

In [53]:
print('Accuracy is: '+str(1.0*sum((result['Y_wkd_test']==1)==(result['dl_predict']>0.5))/len(result['dl_predict'])))

Accuracy is: 0.9702602230483272


## Task 2. Classify all days of the week
Train a neural network against the origial categorical label. Use 5 layers of 40,15,5 and 7 (outputs, representing probabilities for a current input to correspond to each of the weekdays) neurons over the training sample, evaluating its performance over the test sample (use 'categorical_accurary'). Report the acheived accuracy (categorical) over the test sample.

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 200 epochs, batch_size=20

In [54]:
dim = X_train.shape[1]
dim

100

In [55]:
np.random.seed(2019)
model2 = Sequential()
model2.add(Dense(units=40, input_dim=dim, activation='relu'))
model2.add(Dense(units=15, activation='relu'))
model2.add(Dense(units=5, activation='relu'))
model2.add(Dense(units=7, activation='sigmoid'))

In [56]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=20, verbose=2)
scores2 = model2.evaluate(X_test, y_test, verbose=0)
prob2 = model2.predict(X_test)

Epoch 1/200
20/20 - 0s - loss: 0.6929 - categorical_accuracy: 0.1450 - val_loss: 0.6573 - val_categorical_accuracy: 0.1413
Epoch 2/200
20/20 - 0s - loss: 0.6189 - categorical_accuracy: 0.1600 - val_loss: 0.5867 - val_categorical_accuracy: 0.1747
Epoch 3/200
20/20 - 0s - loss: 0.5637 - categorical_accuracy: 0.1700 - val_loss: 0.5387 - val_categorical_accuracy: 0.1413
Epoch 4/200
20/20 - 0s - loss: 0.5191 - categorical_accuracy: 0.1450 - val_loss: 0.4986 - val_categorical_accuracy: 0.1487
Epoch 5/200
20/20 - 0s - loss: 0.4837 - categorical_accuracy: 0.1500 - val_loss: 0.4688 - val_categorical_accuracy: 0.1413
Epoch 6/200
20/20 - 0s - loss: 0.4588 - categorical_accuracy: 0.1525 - val_loss: 0.4490 - val_categorical_accuracy: 0.1413
Epoch 7/200
20/20 - 0s - loss: 0.4429 - categorical_accuracy: 0.1675 - val_loss: 0.4369 - val_categorical_accuracy: 0.1413
Epoch 8/200
20/20 - 0s - loss: 0.4327 - categorical_accuracy: 0.1825 - val_loss: 0.4288 - val_categorical_accuracy: 0.1524
Epoch 9/200
20/2

Epoch 68/200
20/20 - 0s - loss: 0.2662 - categorical_accuracy: 0.5700 - val_loss: 0.2725 - val_categorical_accuracy: 0.5688
Epoch 69/200
20/20 - 0s - loss: 0.2664 - categorical_accuracy: 0.5425 - val_loss: 0.2721 - val_categorical_accuracy: 0.5502
Epoch 70/200
20/20 - 0s - loss: 0.2652 - categorical_accuracy: 0.5450 - val_loss: 0.2720 - val_categorical_accuracy: 0.5688
Epoch 71/200
20/20 - 0s - loss: 0.2647 - categorical_accuracy: 0.5250 - val_loss: 0.2690 - val_categorical_accuracy: 0.5651
Epoch 72/200
20/20 - 0s - loss: 0.2616 - categorical_accuracy: 0.5475 - val_loss: 0.2690 - val_categorical_accuracy: 0.5688
Epoch 73/200
20/20 - 0s - loss: 0.2607 - categorical_accuracy: 0.5600 - val_loss: 0.2681 - val_categorical_accuracy: 0.5688
Epoch 74/200
20/20 - 0s - loss: 0.2587 - categorical_accuracy: 0.5750 - val_loss: 0.2683 - val_categorical_accuracy: 0.5688
Epoch 75/200
20/20 - 0s - loss: 0.2603 - categorical_accuracy: 0.5600 - val_loss: 0.2717 - val_categorical_accuracy: 0.5316
Epoch 76

Epoch 134/200
20/20 - 0s - loss: 0.2251 - categorical_accuracy: 0.5750 - val_loss: 0.2434 - val_categorical_accuracy: 0.5836
Epoch 135/200
20/20 - 0s - loss: 0.2242 - categorical_accuracy: 0.5775 - val_loss: 0.2436 - val_categorical_accuracy: 0.5725
Epoch 136/200
20/20 - 0s - loss: 0.2232 - categorical_accuracy: 0.6000 - val_loss: 0.2430 - val_categorical_accuracy: 0.5725
Epoch 137/200
20/20 - 0s - loss: 0.2226 - categorical_accuracy: 0.5775 - val_loss: 0.2434 - val_categorical_accuracy: 0.6059
Epoch 138/200
20/20 - 0s - loss: 0.2231 - categorical_accuracy: 0.5850 - val_loss: 0.2432 - val_categorical_accuracy: 0.6022
Epoch 139/200
20/20 - 0s - loss: 0.2240 - categorical_accuracy: 0.6000 - val_loss: 0.2439 - val_categorical_accuracy: 0.5948
Epoch 140/200
20/20 - 0s - loss: 0.2224 - categorical_accuracy: 0.6025 - val_loss: 0.2441 - val_categorical_accuracy: 0.5725
Epoch 141/200
20/20 - 0s - loss: 0.2233 - categorical_accuracy: 0.5975 - val_loss: 0.2425 - val_categorical_accuracy: 0.5874


Epoch 200/200
20/20 - 0s - loss: 0.2018 - categorical_accuracy: 0.6400 - val_loss: 0.2376 - val_categorical_accuracy: 0.6208


In [57]:
from sklearn.metrics import accuracy_score
print('Accuracy is: %f' % accuracy_score(np.argmax(y_test,axis=-1), prob2.argmax(axis = 1)))

Accuracy is: 0.620818


## part 2

Use the same datasets as in the Advanced NN lab and further train NN with different architectures

## Task 1

Try the facial recognition task from NN lab 2 with different model specifications and report model accuracy:
1. Try different dropouts - 0.2, 0.3, 0.4
2. Try different convolution windows: (3$\times$3, 10$\times$10)

In [65]:
# load the data

from sklearn.datasets import fetch_lfw_people 
from keras.datasets import mnist

lfw_people = fetch_lfw_people(min_faces_per_person = 70, resize = 0.4) 
  
# the images arrays to find the shapes (for plotting) 
n_samples, h, w = lfw_people.images.shape 
  
# Instead of providing 2D data, X has data already in the form  of a vector that 
# is required in this approach. 
X = lfw_people.data 
n_features = X.shape[1] 

y = lfw_people.target 
target_names = lfw_people.target_names 
n_classes = target_names.shape[0] 

In [66]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [67]:
X_train.shape

(60000, 28, 28)

In [68]:
y_train.shape

(60000,)

In [69]:
# Preprocess the NMIST data, but maintain datastructure for CNN.

num_pixels = X_train.shape[1] * X_train.shape[2]
X_train_pp = (X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')) / 255
X_test_pp = (X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')) / 255


# Keras needs the labels as categorical metrics.
y_train_pp = np_utils.to_categorical(y_train)
y_test_pp = np_utils.to_categorical(y_test)
num_classes = y_test_pp.shape[1]

In [70]:
## model

np.random.seed(1066)
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Dropout(0.20))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=5, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/5
300/300 - 8s - loss: 0.2815 - accuracy: 0.9183 - val_loss: 0.1054 - val_accuracy: 0.9692
Epoch 2/5
300/300 - 8s - loss: 0.0924 - accuracy: 0.9734 - val_loss: 0.0741 - val_accuracy: 0.9761
Epoch 3/5
300/300 - 8s - loss: 0.0636 - accuracy: 0.9814 - val_loss: 0.0543 - val_accuracy: 0.9826
Epoch 4/5
300/300 - 9s - loss: 0.0482 - accuracy: 0.9855 - val_loss: 0.0500 - val_accuracy: 0.9830
Epoch 5/5
300/300 - 8s - loss: 0.0398 - accuracy: 0.9879 - val_loss: 0.0461 - val_accuracy: 0.9843


In [71]:
## model

np.random.seed(1066)
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Dropout(0.30))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=5, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/5
300/300 - 9s - loss: 0.2864 - accuracy: 0.9166 - val_loss: 0.1060 - val_accuracy: 0.9702
Epoch 2/5
300/300 - 9s - loss: 0.1028 - accuracy: 0.9695 - val_loss: 0.0685 - val_accuracy: 0.9780
Epoch 3/5
300/300 - 9s - loss: 0.0717 - accuracy: 0.9783 - val_loss: 0.0611 - val_accuracy: 0.9798
Epoch 4/5
300/300 - 8s - loss: 0.0565 - accuracy: 0.9831 - val_loss: 0.0470 - val_accuracy: 0.9833
Epoch 5/5
300/300 - 9s - loss: 0.0465 - accuracy: 0.9858 - val_loss: 0.0481 - val_accuracy: 0.9835


In [72]:
## model

np.random.seed(1066)
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Dropout(0.40))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=5, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/5
300/300 - 9s - loss: 0.2828 - accuracy: 0.9187 - val_loss: 0.1031 - val_accuracy: 0.9700
Epoch 2/5
300/300 - 9s - loss: 0.0992 - accuracy: 0.9704 - val_loss: 0.0672 - val_accuracy: 0.9782
Epoch 3/5
300/300 - 9s - loss: 0.0733 - accuracy: 0.9775 - val_loss: 0.0497 - val_accuracy: 0.9839
Epoch 4/5
300/300 - 9s - loss: 0.0583 - accuracy: 0.9822 - val_loss: 0.0450 - val_accuracy: 0.9837
Epoch 5/5
300/300 - 9s - loss: 0.0479 - accuracy: 0.9855 - val_loss: 0.0455 - val_accuracy: 0.9840


In [74]:
## model

np.random.seed(1066)
model = Sequential()
model.add(Conv2D(32, (10, 10), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Dropout(0.20))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=5, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/5
300/300 - 10s - loss: 0.2445 - accuracy: 0.9290 - val_loss: 0.0810 - val_accuracy: 0.9747
Epoch 2/5
300/300 - 9s - loss: 0.0738 - accuracy: 0.9781 - val_loss: 0.0496 - val_accuracy: 0.9845
Epoch 3/5
300/300 - 9s - loss: 0.0526 - accuracy: 0.9841 - val_loss: 0.0395 - val_accuracy: 0.9873
Epoch 4/5
300/300 - 9s - loss: 0.0410 - accuracy: 0.9872 - val_loss: 0.0366 - val_accuracy: 0.9875
Epoch 5/5
300/300 - 9s - loss: 0.0334 - accuracy: 0.9891 - val_loss: 0.0342 - val_accuracy: 0.9885


In [73]:
## model

np.random.seed(1066)
model = Sequential()
model.add(Conv2D(32, (10, 10), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Dropout(0.30))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=5, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/5
300/300 - 10s - loss: 0.2544 - accuracy: 0.9257 - val_loss: 0.0755 - val_accuracy: 0.9767
Epoch 2/5
300/300 - 9s - loss: 0.0775 - accuracy: 0.9768 - val_loss: 0.0627 - val_accuracy: 0.9795
Epoch 3/5
300/300 - 9s - loss: 0.0565 - accuracy: 0.9820 - val_loss: 0.0417 - val_accuracy: 0.9868
Epoch 4/5
300/300 - 9s - loss: 0.0436 - accuracy: 0.9867 - val_loss: 0.0373 - val_accuracy: 0.9879
Epoch 5/5
300/300 - 9s - loss: 0.0371 - accuracy: 0.9885 - val_loss: 0.0343 - val_accuracy: 0.9885


In [75]:
## model

np.random.seed(1066)
model = Sequential()
model.add(Conv2D(32, (10, 10), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Dropout(0.40))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=5, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/5
300/300 - 10s - loss: 0.2519 - accuracy: 0.9268 - val_loss: 0.0693 - val_accuracy: 0.9797
Epoch 2/5
300/300 - 10s - loss: 0.0780 - accuracy: 0.9759 - val_loss: 0.0512 - val_accuracy: 0.9833
Epoch 3/5
300/300 - 10s - loss: 0.0566 - accuracy: 0.9829 - val_loss: 0.0344 - val_accuracy: 0.9891
Epoch 4/5
300/300 - 10s - loss: 0.0462 - accuracy: 0.9854 - val_loss: 0.0349 - val_accuracy: 0.9885
Epoch 5/5
300/300 - 10s - loss: 0.0378 - accuracy: 0.9877 - val_loss: 0.0295 - val_accuracy: 0.9898


## Task 2

1. Try different word count in the next-word prediction task with a)8 and b)12 previous words and report accuracy for each. Use the same Sherlock data.

2. Try different LSTM architectures and comment on model performace for each. 
    - Add a dropout layer with 0.25
    - Change number of neurons to a) 64 b) 256 neurons
    
    Train for 10 epochs 

In [18]:
path = 'sherlock.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 581889


1.Try different word count in the next-word prediction task with a)8 and b)12 previous words and report accuracy for each. Use the same Sherlock data.

In [85]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [86]:
# unique sorted words list
unique_words = np.unique(words)

# dictionary(<key: value>) with each word form the unique_words list as key and its corresponding position as value.
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [87]:
WORD_LENGTH = 8
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print('sequence:', prev_words[10101])
print('next word in sequence:', next_words[10101])

sequence: ['of', 'twenty', 'one', 'years', 'are', 'eligible', 'apply', 'in']
next word in sequence: person


In [88]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [89]:
X[0].shape

(8, 8201)

In [131]:
model3 = Sequential()
model3.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model3.add(Dense(len(unique_words)))
model3.add(Activation('softmax'))

In [132]:
# training - just 2 epochs - should take around 5-7 mins for 2 epochs

np.random.seed(2002)
optimizer = RMSprop(lr=0.01)
model3.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model3.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

Epoch 1/2
811/811 [==============================] - 205s 252ms/step - loss: 6.3345 - accuracy: 0.0831 - val_loss: 7.0238 - val_accuracy: 0.0992
Epoch 2/2
811/811 [==============================] - 217s 268ms/step - loss: 5.6421 - accuracy: 0.1474 - val_loss: 7.7877 - val_accuracy: 0.0972


In [97]:
WORD_LENGTH = 12
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print('sequence:', prev_words[10101])
print('next word in sequence:', next_words[10101])

sequence: ['of', 'twenty', 'one', 'years', 'are', 'eligible', 'apply', 'in', 'person', 'on', 'monday', 'at']
next word in sequence: eleven


In [98]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [99]:
X[0].shape

(12, 8201)

In [100]:
model4 = Sequential()
model4.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model4.add(Dense(len(unique_words)))
model4.add(Activation('softmax'))

In [101]:
# training - just 2 epochs - should take around 5-7 mins for 2 epochs

np.random.seed(2002)
optimizer = RMSprop(lr=0.01)
model4.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model4.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

Epoch 1/2
811/811 [==============================] - 210s 256ms/step - loss: 6.3177 - accuracy: 0.0831 - val_loss: 7.0741 - val_accuracy: 0.1027
Epoch 2/2
811/811 [==============================] - 216s 267ms/step - loss: 5.6304 - accuracy: 0.1486 - val_loss: 7.9565 - val_accuracy: 0.1082


Part 2 Try different LSTM architectures and comment on model performace for each.
Add a dropout layer with 0.25
Change number of neurons to a) 64 b) 256 neurons
Train for 10 epochs

In [122]:
# Preprocess the NMIST data, but maintain datastructure for CNN.

num_pixels = X_train.shape[1] * X_train.shape[2]
X_train_pp = (X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')) / 255
X_test_pp = (X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')) / 255


# Keras needs the labels as categorical metrics.
y_train_pp = np_utils.to_categorical(y_train)
y_test_pp = np_utils.to_categorical(y_test)
num_classes = y_test_pp.shape[1]

In [123]:
## model

np.random.seed(1066)
model5 = Sequential()
model5.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model5.add(MaxPooling2D(pool_size=(2, 2)))


model5.add(Dropout(0.25))
model5.add(Flatten())
model5.add(Dense(64, activation='relu'))
model5.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=10, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/10
300/300 - 10s - loss: 0.2752 - accuracy: 0.9239 - val_loss: 0.0954 - val_accuracy: 0.9698
Epoch 2/10
300/300 - 10s - loss: 0.0862 - accuracy: 0.9746 - val_loss: 0.0536 - val_accuracy: 0.9827
Epoch 3/10
300/300 - 9s - loss: 0.0616 - accuracy: 0.9814 - val_loss: 0.0483 - val_accuracy: 0.9843
Epoch 4/10
300/300 - 9s - loss: 0.0496 - accuracy: 0.9844 - val_loss: 0.0403 - val_accuracy: 0.9856
Epoch 5/10
300/300 - 10s - loss: 0.0422 - accuracy: 0.9870 - val_loss: 0.0442 - val_accuracy: 0.9850
Epoch 6/10
300/300 - 10s - loss: 0.0379 - accuracy: 0.9876 - val_loss: 0.0401 - val_accuracy: 0.9862
Epoch 7/10
300/300 - 10s - loss: 0.0317 - accuracy: 0.9903 - val_loss: 0.0311 - val_accuracy: 0.9884
Epoch 8/10
300/300 - 10s - loss: 0.0289 - accuracy: 0.9911 - val_loss: 0.0350 - val_accuracy: 0.9884
Epoch 9/10
300/300 - 10s - loss: 0.0254 - accuracy: 0.9919 - val_loss: 0.0312 - val_accuracy: 0.9901
Epoch 10/10
300/300 - 10s - loss: 0.0217 - accuracy: 0.9931 - val_loss: 0.0314 - val_accuracy

In [128]:
accuracy_score(y_test, model5.predict_classes(X_test_pp))

0.9899

In [120]:
## model

np.random.seed(1066)
model6 = Sequential()
model6.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model6.add(MaxPooling2D(pool_size=(2, 2)))


model6.add(Dropout(0.25))
model6.add(Flatten())
model6.add(Dense(256, activation='relu'))
model6.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
model6.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model6.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=10, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/10
300/300 - 9s - loss: 0.2184 - accuracy: 0.9361 - val_loss: 0.0666 - val_accuracy: 0.9799
Epoch 2/10
300/300 - 9s - loss: 0.0676 - accuracy: 0.9790 - val_loss: 0.0484 - val_accuracy: 0.9850
Epoch 3/10
300/300 - 9s - loss: 0.0461 - accuracy: 0.9859 - val_loss: 0.0348 - val_accuracy: 0.9882
Epoch 4/10
300/300 - 9s - loss: 0.0372 - accuracy: 0.9885 - val_loss: 0.0336 - val_accuracy: 0.9879
Epoch 5/10
300/300 - 9s - loss: 0.0280 - accuracy: 0.9909 - val_loss: 0.0296 - val_accuracy: 0.9900
Epoch 6/10
300/300 - 9s - loss: 0.0228 - accuracy: 0.9927 - val_loss: 0.0307 - val_accuracy: 0.9885
Epoch 7/10
300/300 - 10s - loss: 0.0198 - accuracy: 0.9936 - val_loss: 0.0296 - val_accuracy: 0.9898
Epoch 8/10
300/300 - 10s - loss: 0.0164 - accuracy: 0.9948 - val_loss: 0.0324 - val_accuracy: 0.9893
Epoch 9/10
300/300 - 10s - loss: 0.0131 - accuracy: 0.9958 - val_loss: 0.0283 - val_accuracy: 0.9907
Epoch 10/10
300/300 - 11s - loss: 0.0120 - accuracy: 0.9960 - val_loss: 0.0318 - val_accuracy: 0.

In [127]:
accuracy_score(y_test, model6.predict_classes(X_test_pp))

0.9895

In [129]:
#comment on both models: Accuracy decreased very slightly